In [2]:
import os
import json 
import os
import json
from tqdm import tqdm
import pickle

In [3]:
T = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!?{~}(^)[_]@#$,./&-*++="

def decode(e):
    f = []
    g = len(e)
    for c in range(0, g, 2):
        d = {}
        b = T.index(e[c])
        a = T.index(e[c + 1])
        if a > 63:
            d["promotion"] = "qnrbkp"[int((a - 64) / 3)]
            a = b + (-8 if b < 16 else 8) + (a - 1) % 3 - 1
        if b > 75:
            d["drop"] = "qnrbkp"[b - 79]
        else:
            d["from"] = T[b % 8] + str(int(b / 8) + 1)
        d["to"] = T[a % 8] + str(int(a / 8) + 1)
        f.append(d)
    return f

In [4]:
class TreeNode:
    def __init__(self, move=None):
        self.move = move
        self.children = {}
        self.count = 1

    def add_child(self, move):
        if move not in self.children:
            self.children[move] = TreeNode(move)
        else:
            self.children[move].count += 1
        return self.children[move]

    def get_child(self, move):
        return self.children.get(move)

    def __repr__(self):
        return f"Move: {self.move}, Count: {self.count}"

class ChessOpeningTree:
    def __init__(self):
        self.root = TreeNode("START")

    def add_game(self, moves):
        current_node = self.root
        for move in moves:
            move_str = f"{move['from']}->{move['to']}"
            current_node = current_node.add_child(move_str)

    def __repr__(self):
        return str(self.root)

In [5]:
# Initialize the tree
opening_tree = ChessOpeningTree()

# Path to the directory containing the game files
game_files_dir = "../Data/Games"

# List all the game files
game_files = os.listdir(game_files_dir)

skipped = 0
# Loop through each game file
for file in tqdm(game_files):
    with open(os.path.join(game_files_dir, file)) as f:
        game_json = json.load(f)
    if "game" not in game_json:
        skipped += 1
        continue   
    # Extract the first 15 moves from each game
    moves = decode(game_json["game"]["moveList"])[:20]
    opening_tree.add_game(moves)

100%|██████████| 144629/144629 [00:41<00:00, 3516.57it/s]


In [7]:
with open("../Data/opening_tree.pkl", "wb") as f:
    pickle.dump(opening_tree, f)

In [14]:
"c2->c4" in opening_tree.root.children

True

In [23]:
opening_tree.root.children["c2->c4"].count

8136

In [30]:
def move_counts_from_graphml(moveList, tree):
    # Load the graph from the saved GraphML file    
    move_stats = {}
    current_node = "START"
    cur = opening_tree.root
    depth = 0

    for move in moveList:
        move_str = f"{move['from']}->{move['to']}"
        if move_str in cur.children:
            move_stats[move_str] = {}
            move_stats[move_str]["count"] = int(cur.children[move_str].count)  
            move_stats[move_str]["rank"] = sorted(cur.children, key=lambda x: cur.children[x].count, reverse=True).index(move_str) + 1
            cur = cur.children[move_str]
            
        else:
            move_stats[move_str] = {}
            move_stats[move_str]["count"] = 0
            move_stats[move_str]["rank"] = -1

            break
        depth += 1
    
    return move_stats


stafford_gambit_moves = [
    {'from': 'e2', 'to': 'e4'},
    {'from': 'e7', 'to': 'e5'},
    {'from': 'g1', 'to': 'f3'},
    {'from': 'g8', 'to': 'f6'},
]

# Given sequence of moves in algebraic notation
moves_sequence = [
    ('e2', 'e4'), ('e7', 'e5'),
    ('g1', 'f3'), ('b8', 'c6'),
    ('f1', 'b5'), ('g8', 'f6'),
    ('e1', 'g1'), ('f6', 'e4'),
    ('d2', 'd4'), ('e4', 'd6'),
    ('d4', 'e5'), ('d6', 'b5'),
    ('a2', 'a4'), ('b5', 'd4'),
    ('f3', 'd4'), ('c6', 'd4'),
    ('d1', 'd4'), ('d7', 'd5'),
    ('e5', 'd6'), ('d8', 'd6'),  # e.p. stands for en passant, but it's not a separate move.
    ('d4', 'e4'), ('d6', 'e6'),
    ('e4', 'd4'), ('e6', 'd6'),
    ('d4', 'e4'), ('d6', 'e6'),
    ('e4', 'd4'), ('d6', 'd6')
]

# Convert to the specified move list format
berlin = [{'from': start, 'to': end} for start, end in moves_sequence]
move_counts_from_graphml(berlin,opening_tree)


{'e2->e4': {'count': 68467, 'rank': 1},
 'e7->e5': {'count': 12711, 'rank': 2},
 'g1->f3': {'count': 11013, 'rank': 1},
 'b8->c6': {'count': 9128, 'rank': 1},
 'f1->b5': {'count': 4103, 'rank': 1},
 'g8->f6': {'count': 643, 'rank': 2},
 'e1->g1': {'count': 311, 'rank': 1},
 'f6->e4': {'count': 288, 'rank': 1},
 'd2->d4': {'count': 140, 'rank': 2},
 'e4->d6': {'count': 138, 'rank': 1},
 'd4->e5': {'count': 46, 'rank': 2},
 'd6->b5': {'count': 45, 'rank': 1},
 'a2->a4': {'count': 45, 'rank': 1},
 'b5->d4': {'count': 35, 'rank': 1},
 'f3->d4': {'count': 35, 'rank': 1},
 'c6->d4': {'count': 11, 'rank': 2},
 'd1->d4': {'count': 11, 'rank': 1},
 'd7->d5': {'count': 6, 'rank': 1},
 'e5->d6': {'count': 5, 'rank': 1},
 'd8->d6': {'count': 5, 'rank': 1},
 'd4->e4': {'count': 0, 'rank': -1}}